<a href="https://colab.research.google.com/github/isavida/football-task/blob/test-yolo-tracking/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GIT configuration

In [ ]:
!git checkout main

fatal: not a git repository (or any of the parent directories): .git


Clone

In [2]:
!git config --global user.email "isabel.vidaurre@hotmail.com"
!git config --global user.name "isavida"

In [3]:
!git clone https://@github.com/isavida/football-task.git

Cloning into 'football-task'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), done.


Commit

In [4]:
%cd football-task

/content/football-task


In [7]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   data/clip_1.mp4
	new file:   data/clip_2.mp4
	new file:   data/clip_3.mp4



In [5]:
!git add data/

In [6]:
!git commit -m "Add input data"

[main 1289579] Add input data
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 data/clip_1.mp4
 create mode 100644 data/clip_2.mp4
 create mode 100644 data/clip_3.mp4


In [7]:
!git push origin main

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 19.91 MiB | 14.99 MiB/s, done.
Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/isavida/football-task.git
   7a0cc12..1289579  main -> main


# Introduction
The task consists of taking as input a video footage and output:


*   A JSONL file that contains a row for each 5th frame with the number of players from home-team and away-team, numbers of referees and position of the ball.
*   A video with the team players with the bounding boxes of detections
classified according to their team and the annotation of the ball when detected


It is decided to do the player detection task first, as the data for the jsonl file can be obtained from these detections.



In [ ]:
!pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO

In [ ]:
#load a model and perform tracking
model = YOLO('yolov8s.pt')

results = model.track(source="football_data/clip_1.mp4", save = True, name="yolov8s")